# Chiral solutions for Type-II Dirac seesaw with a Dark Symmetry

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

## Load full solutions

In [2]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(148097, 5)

Check that solutions are unique

In [3]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(148097, 6)

In [4]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [5]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

95358

## Check phenomenological conditions
$$ \nu+m+s=0$$

In [6]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [6]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    triplet=False
    for x in ll:
        if ll.count(x)>1:
            if ll.count(x)>2:
                triplet=True
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2 and triplet:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (.ν,ν,...)
    '''
    fs=[]
    mrp=[]
    νrp=[]
    for x in ll:
        if ll.count(x)==1:
            fs.append(x)
        elif ll.count(x)==3:
            mrp.append(x)
        elif ll.count(x)>3:
            mrp.append(x)
            fs.append(x)
        elif ll.count(x)==2:
            νrp.append(x)
    mrp=sorted(np.unique(mrp))
    νrp=sorted(np.unique(νrp))
    fs=sorted(np.unique(fs))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for m in mrp:
        for ν in νrp:
            #if (mrp[0]+ν in [sum(k)  for k  in itertools.combinations(fs,2)]+list(2*np.array(fs))):
            s=-m-ν# ,fs,[sum(k)  for k  in itertools.combinations(fs,2)],list(2*np.array(fs)))
            nom=mrp.copy()
            kk=nom.remove(m)
            noν=νrp.copy()
            kk=noν.remove(ν)
            massless=fs+nom+noν
            massless=extract_Dirac_and_Majorana(massless,s)
            slt={'[m,ν,s]':[m,ν,s],'massless':massless}
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)            
            sltn.append(slt)
    return sltn

#assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
#assert best_higgs_singlet([1,-11,9])==(2, [])
#assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]
#assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

In [7]:
def tree_level_ii(ll):
    '''
    At least a triplet and a doublet is guaranteed
    (ν,ν,...)
    '''
    fs=[]
    νrp=[]
    for x in np.unique(ll):
        if ll.count(x)==2 or ll.count(x)==3:
            νrp.append(x)
    νrp=sorted(np.unique(νrp))
    #print(len(mrp),len(νrp),ll)
    sltn=[]
    for ν in νrp:
        s=-ν
        fs=list(np.unique([x for x in ll if x!=ν]))
        massless=extract_Dirac_and_Majorana(fs,s)
        if len(massless)==0:
            slt={'[ν,s]':[ν,s]}
            sltn.append(slt)
    if len(sltn)>0:
        return sltn
    else:
        return np.nan

In [8]:
ll=[6, -5, -5, 3, 2, -1]
assert tree_level_ii(ll)==[{'[ν,s]': [-5, 5]}]

### Initialize filtered solutions

In [9]:
cl=ds.copy()

Find solutions that satisfy the condition

In [10]:
cl['tree_level']=cl['solution'].apply(tree_level_ii)

In [11]:
cl=cl.dropna(subset=['tree_level']).reset_index(drop=True)

In [12]:
cl.shape

(979, 6)

In [234]:
#cl=cl[cl['n']==9].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level
0,"[1, -1, 0, 1]","[-2, 0, 2, -2]","[1, 1, 1, -5, -6, 10, 10, 10, -11, -11]",1,10,"[{'[ν,s]': [-11, 11]}]"


In [235]:
cl.shape

(979, 6)

In [238]:
def rank(l,ν):
    '''
    Obtain a list of count of repeated charges, 
    * if the list have an even length → Pairs match
    * if the list have an odd length → Pairs does not match → massless fermions
    '''
    #Remove ν
    l=[x for x in  l if x!=ν ]
    s=-ν
    #Remove Majorana
    xx=(np.unique(l)*2-s)
    xx=(xx[xx!=0]+s)/2
    xx=(np.unique(xx)*2+s)
    xx=(xx[xx!=0]-s)/2
    return len([l.count(x) for x in xx if l.count(x)!=1])%2

assert( rank( l=[1, -2, -3, 5, 5, -6], ν=5 )) == 0
assert( rank(l=[1, -2, 3, 4, 6, -7, -7, -7, 9],ν=-7 )) == 0
assert( rank( l=[1, 1, 2, 2, 3, -5, -6, -6, 8],ν=2 ))==1

cl['ν']=cl['tree_level'].apply(lambda l: [d.get('[ν,s]')[0]  for d in l  ]).str[0]
cl=cl[cl.apply(lambda r: rank( r['solution'], -r['ν'] )==0,axis='columns')==0].reset_index(drop=True).drop('ν',axis='columns')

In [239]:
cl.shape

(957, 6)

In [240]:
kk=cl[cl['n']==10]
kk[kk['solution'].apply(lambda l: np.abs(l).max())<9]

,l,k,solution,gcd,n,tree_level
212,"[-2, 0, 2, 1]","[-5, -4, 4, 3]","[1, 1, -2, -2, -2, -2, 3, 4, 4, -5]",1,10,"[{'[ν,s]': [1, -1]}]"
260,"[-2, -1, 0, 1]","[-5, -6, -4, 1]","[1, -2, -2, 3, 4, -5, -5, 7, 7, -8]",1,10,"[{'[ν,s]': [-5, 5]}, {'[ν,s]': [7, -7]}]"
433,"[2, 3, -1, 1]","[-6, -5, -6, -5]","[1, -2, -2, 3, 3, -4, -4, 6, 6, -7]",1,10,"[{'[ν,s]': [6, -6]}]"


Reorder the filtered solutions

In [245]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl.sort_values(['n','smax']).reset_index(drop=True)
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,"[{'[ν,s]': [5, -5]}]",6


### Flag equivalent solutions

In [252]:
def characterize_solution(cl,i,j=0):
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    cl.loc[i,'trptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>2] )
    return cl

Example: massless solutions

In [253]:
for i in cl.index:
    cl=characterize_solution(cl,i,0)

In [278]:
cl.dropna()[:1]

,l,k,solution,gcd,n,tree_level,smax,rptd,trptd
0,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,"[{'[ν,s]': [5, -5]}]",6,1,0


### Add solutions with massless fermions that get masses with some $S'$

In [255]:
for k  in ['rptd','trptd']:
    cl[k]=cl[k].astype(int)

In [256]:
cl=cl.sort_values(['n','smax','rptd','trptd']).reset_index(drop=True)

## Build the final table dropping out equivalent solutions 

In [266]:
fl=pd.DataFrame()

In [267]:
fl=fl.append( cl[ cl['n']==6 ].drop_duplicates(subset=['rptd','trptd']).reset_index(drop=True) )

In [268]:
fl=fl.append(cl[ cl['n']==7 ].drop_duplicates(subset=['rptd','trptd']).reset_index(drop=True) )

In [269]:
fl=fl.append(cl[ cl['n']==8 ].drop_duplicates(subset=['rptd','trptd']).reset_index(drop=True) )

In [270]:
fl=fl.append( cl[ cl['n']==9 ].drop_duplicates(subset=['rptd','trptd']).reset_index(drop=True) )

In [271]:
fl=fl.reset_index(drop=True)

In [272]:
fl

,l,k,solution,gcd,n,tree_level,smax,rptd,trptd
0,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,6,"[{'[ν,s]': [5, -5]}]",6,1,0
1,"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,9,"[{'[ν,s]': [-7, 7]}]",9,1,1
2,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,"[{'[ν,s]': [9, -9]}]",10,3,1


In [277]:
cl[cl['n']==10].drop_duplicates(subset=['rptd','trptd'])

,l,k,solution,gcd,n,tree_level,smax,rptd,trptd
26,"[-2, 0, 2, 1]","[-5, -4, 4, 3]","[1, 1, -2, -2, -2, -2, 3, 4, 4, -5]",1,10,"[{'[ν,s]': [1, -1]}]",5,3,1
27,"[2, 3, -1, 1]","[-6, -5, -6, -5]","[1, -2, -2, 3, 3, -4, -4, 6, 6, -7]",1,10,"[{'[ν,s]': [6, -6]}]",7,4,0
28,"[-2, -1, 0, 1]","[-5, -6, -4, 1]","[1, -2, -2, 3, 4, -5, -5, 7, 7, -8]",1,10,"[{'[ν,s]': [-5, 5]}, {'[ν,s]': [7, -7]}]",8,3,0
35,"[5, 0, -2, 2]","[-6, -5, 1, 0]","[1, 1, 5, 5, 5, -6, -6, -6, -9, 10]",1,10,"[{'[ν,s]': [1, -1]}]",10,3,2
41,"[2, -2, 2, 3]","[-6, -5, -3, 1]","[1, -2, -3, 5, 6, -8, -9, 11, 11, -12]",1,10,"[{'[ν,s]': [11, -11]}]",12,1,0
49,"[-2, 4, 0, -3]","[-2, 0, -1, 1]","[1, 1, -2, -2, -4, 6, -10, 11, 12, -13]",1,10,"[{'[ν,s]': [-2, 2]}]",13,2,0
50,"[-4, -5, -4, -2]","[-4, 3, -5, 0]","[3, 4, 4, 4, 4, -5, -8, -8, -11, 13]",8,10,"[{'[ν,s]': [-8, 8]}]",13,2,1


In [279]:
def getit(it,dk):
    it['ν']=dk.get('[ν,s]')[0]
    it['s']=dk.get('[ν,s]')[1]
    it['massless']=0
    return it
    
tm=pd.DataFrame()
fl=fl.sort_values(['n','smax']).reset_index(drop=True)
for i in fl.index:
    it=fl.loc[i].to_dict()
    k=[d for d in fl.loc[i,'tree_level']]
    if len(k)>0:
        dk=k[0]
        it=getit(it,dk)
        tm=tm.append(it,ignore_index=True)

In [280]:
tm

,gcd,k,l,massless,n,rptd,s,smax,solution,tree_level,trptd,ν
0,1.0,"[-2, 0]","[-1, 1]",0.0,6.0,1.0,-5.0,6.0,"[1, -2, -3, 5, 5, -6]","[{'[ν,s]': [5, -5]}]",0.0,5.0
1,1.0,"[-3, -1, -2, -1]","[-2, -3, 1, -1]",0.0,9.0,1.0,7.0,9.0,"[1, -2, 3, 4, 6, -7, -7, -7, 9]","[{'[ν,s]': [-7, 7]}]",1.0,-7.0
2,1.0,"[-1, 1, 0, -1]","[-2, 0, 2]",0.0,9.0,3.0,-9.0,10.0,"[1, 1, -4, -5, 9, 9, 9, -10, -10]","[{'[ν,s]': [9, -9]}]",1.0,9.0


In [281]:
for k in ['n','gcd','ν','s','rptd','trptd']:
    tm[k]=tm[k].astype(int)
tm['nsmax']=tm.apply(lambda row: np.abs( row['solution'] ).max(),axis='columns' )
tm=tm.sort_values(['n','nsmax']).reset_index(drop=True)

#Delete equivalent solutions
#tm['Q']=True
#sltns={'[3, -4, 8, -9]': [3, -4, -6, -6, 7, 7, 8, -9],}
#for k in sltns.keys():
#    iq=tm[((tm['solution'].astype(str)==str(sltns[k])) &  (tm['massless'].astype(str)==k)
#          )].index[0] #unstable dark matter
#    tm.loc[iq,'Q']=False

#tm=tm[tm['Q']].reset_index(drop=True).drop('Q',axis='columns')
tm[['n','l','k','solution','gcd','ν','s','massless','rptd','trptd']
  ].sort_values(['n','rptd','trptd'])#.loc[[0,4]]#.loc[[0,4]]

,n,l,k,solution,gcd,ν,s,massless,rptd,trptd
0,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,0.0,1,0
1,9,"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,-7,7,0.0,1,1
2,9,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,-9,0.0,3,1


In [282]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless','rptd','trptd']].reset_index(drop=True)
kk['sltn']=kk['solution'].astype(str)
kk.sort_values('n').drop('sltn',axis='columns').reset_index(drop=True)

,n,l,k,solution,gcd,ν,s,massless,rptd,trptd
0,6,"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,0.0,1,0
1,9,"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,-7,7,0.0,1,1
2,9,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,-9,0.0,3,1


In [283]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless']].copy()

In [284]:
import re
def add_boldsymbol(ss):
    if str(ss).find(r'\boldsymbol')==-1:
        return re.sub('(\-*[0-9]+)',r'\\boldsymbol{\1}',str(ss))
    else:
        return ss

In [287]:
kk=tm[['n','l','k','solution','gcd','ν','s','massless']].copy()

for i in kk.index:
    #for s in ['DD','DM','XD','XM']:
    if not kk.loc[i,'massless']:
        print(i)
        kk.loc[i,'n']=add_boldsymbol(kk.loc[i,'n'])
        #kk.loc[i,s]  =add_boldsymbol(kk.loc[i,s])

kkk=kk#[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

0
1
2


In [288]:
cat solutions.tex

\begin{tabular}{llllrrrr}
\toprule
              n &                l &                 k &                           solution & gcd &  ν &  s & massless \\
\midrule
 \boldsymbol{6} &          (-1, 1) &           (-2, 0) &              (1, -2, -3, 5, 5, -6) &   1 &  5 & -5 &      0.0 \\
 \boldsymbol{9} &  (-2, -3, 1, -1) &  (-3, -1, -2, -1) &    (1, -2, 3, 4, 6, -7, -7, -7, 9) &   1 & -7 &  7 &      0.0 \\
 \boldsymbol{9} &       (-2, 0, 2) &    (-1, 1, 0, -1) &  (1, 1, -4, -5, 9, 9, 9, -10, -10) &   1 &  9 & -9 &      0.0 \\
\bottomrule
\end{tabular}


In [289]:
kkk

,n,l,k,solution,gcd,ν,s,massless
0,\boldsymbol{6},"[-1, 1]","[-2, 0]","[1, -2, -3, 5, 5, -6]",1,5,-5,0.0
1,\boldsymbol{9},"[-2, -3, 1, -1]","[-3, -1, -2, -1]","[1, -2, 3, 4, 6, -7, -7, -7, 9]",1,-7,7,0.0
2,\boldsymbol{9},"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,-9,0.0
